In [18]:
import requests
import time

from selenium import webdriver
from selenium.webdriver import Chrome
#from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.common.by import By
#from selenium.webdriver.support.ui import WebDriverWait
#from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome()

class scraper:

    def getURL(url):
        driver.get(url) 

    def getTitle():
        print(driver.title)

    #time.sleep(3)
    #driver.quit()

scraper.getURL('https://www.pickuplimes.com/')
scraper.getTitle()

print("End")

WebDriverException: Message: chrome not reachable
  (Session info: chrome=99.0.4844.84)
Stacktrace:
Backtrace:
	Ordinal0 [0x006E9943+2595139]
	Ordinal0 [0x0067C9F1+2148849]
	Ordinal0 [0x005743F0+1065968]
	Ordinal0 [0x005687C2+1017794]
	Ordinal0 [0x00568FF8+1019896]
	Ordinal0 [0x0056A892+1026194]
	Ordinal0 [0x00564219+999961]
	Ordinal0 [0x00575860+1071200]
	Ordinal0 [0x005CB2D2+1422034]
	Ordinal0 [0x005BB806+1357830]
	Ordinal0 [0x00596086+1204358]
	Ordinal0 [0x00596F96+1208214]
	GetHandleVerifier [0x0088B232+1658114]
	GetHandleVerifier [0x0094312C+2411516]
	GetHandleVerifier [0x0077F261+560433]
	GetHandleVerifier [0x0077E366+556598]
	Ordinal0 [0x0068286B+2173035]
	Ordinal0 [0x006875F8+2192888]
	Ordinal0 [0x006876E5+2193125]
	Ordinal0 [0x006911FC+2232828]
	BaseThreadInitThunk [0x7756FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77AA7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77AA7A4E+238]


In [130]:
# ZOOPLA
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome()  
url = "https://zoopla.co.uk" 
driver.get(url)

time.sleep(5) # Slow it down otherwise it will miss the button 

cookie_button = "Accept all cookies"
driver.find_element(By.LINK_TEXT, cookie_button)

try: 
    frame_id = "gdpr-consent-notice"
    driver.switch_to.frame(frame_id)
    print("Switched to frame")

    #accept_cookies_button = driver.find_element(By.XPATH, '/*[@id="save"]/span[1]/div/span') # XPath for button
    #print("Found Button")
except:
    print("No Cookie Button")
    pass

time.sleep(3)
driver.quit()
#try:
    #accept_cookies_button = driver.find_element(By.Tag_Name, "button") # XPath for button
    #print("Find Button")
    #accept_cookies_button.click() # Click the button 
    # If there isn't a cookie button 
#except: 
    #print("No Cookies Detected")
    #pass

gdpr-consent-notice
Switched to frame


In [20]:
my_list = [1, 2, 3]
print(my_list * 2)


[1, 2, 3, 1, 2, 3]
